### Data Cleaning / Merging

In [34]:
# some imports
import pandas as pd
import numpy as np
import csv
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
# load data (converted from json to csv using yelp git repo converter) (takes a few min)
# data structure: https://www.yelp.co.uk/dataset_challenge/dataset
business = pd.read_csv('yelp_academic_dataset_business.csv')
review = pd.read_csv('yelp_academic_dataset_review.csv')
#user = pd.read_csv('yelp_academic_dataset_user.csv')

c:\Users\Jeremy\Anaconda\lib\site-packages\pandas\io\parsers.py:1170: DtypeWarning: Columns (1,4,7,14,18,21,27,30,43,49,52,62,64,66,70,84,89,92,100) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


In [4]:
# merge datasets
merged = pd.merge(review,business,on='business_id')

In [5]:
subset = merged[['name', 'state', 'city', 'stars_y', 'categories', 'stars_x', 'text']]

In [9]:
# Figure out which attributes are boolean and which are values
cols = [x for x in merged.columns.tolist() if 'attributes' in x]
'''for c in cols:
    u = np.unique(merged[c])
    if len(u) != 3:
        print(c,u)'''

'for c in cols:\n    u = np.unique(merged[c])\n    if len(u) != 3:\n        print(c,u)'

In [10]:
# Handle these columns specially

# First set have a "false" value
special_cols = {
    'attributes.Alcohol': 'none',
    'attributes.BYOB/Corkage': 'no',
    'attributes.Wi-Fi': 'no',
    'attributes.Smoking': 'no'
}

# Second set don't
special_cols_2 = [
    'attributes.Ages Allowed',
    'attributes.Price Range',
    'attributes.Noise Level',
    'attributes.Attire'
]

In [11]:
# Create a single attributes column
subset['attributes'] = ""

# For each column
for c in cols:
    # Don't add for null rows
    b = ~pd.isnull(merged[c])
    
    if c in special_cols:
        b = b&(merged[c] != special_cols[c])
        subset['attributes'][b] += ";" + c[11:] + '=' + merged[c][b].astype(str)
    elif c in special_cols_2:
        subset['attributes'][b] += ";" + c[11:] + '=' + merged[c][b].astype(str)
    else:
        b = b & (merged[c].astype(bool))
        subset['attributes'][b] += ";" + c[11:]
        
# Strip leading semicolon
subset['attributes'][subset['attributes'] != ""] = subset['attributes'][subset['attributes'] != ""].str[1:]

c:\Users\Jeremy\Anaconda\lib\site-packages\IPython\kernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from IPython.kernel.zmq import kernelapp as app
c:\Users\Jeremy\Anaconda\lib\site-packages\IPython\kernel\__main__.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
c:\Users\Jeremy\Anaconda\lib\site-packages\pandas\core\generic.py:3636: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inpla

In [19]:
# clean text of special characters and leading/trailing spaces
subset['text'] = subset['text'].str.replace('\n', ' ').str.strip()
subset['text'] = subset['text'].str.replace('\f', ' ')
subset['text'] = subset['text'].str.replace('\t', ' ')

c:\Users\Jeremy\Anaconda\lib\site-packages\IPython\kernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from IPython.kernel.zmq import kernelapp as app
c:\Users\Jeremy\Anaconda\lib\site-packages\IPython\kernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
c:\Users\Jeremy\Anaconda\lib\site-packages\IPython\kernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See 

In [23]:
# pivot out categories column 
import ast

L = [] # empty list to store categories

for l in subset.categories.unique():
    L.extend(ast.literal_eval(l)) # literal turns string list into list list

print len(set(L)), 'categories'
cat = list(set(L)) # store distinct list

781 categories


In [25]:
subset['categories_new'] = ""
for c in cat:
    b = subset['categories'].str.contains(c)
    subset['categories_new'][b] += ";" + c
    
# Strip leading semicolon
subset['categories_new'][subset['categories_new'] != ""] = subset['categories_new'][subset['categories_new'] != ""].str[1:]

c:\Users\Jeremy\Anaconda\lib\site-packages\IPython\kernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
c:\Users\Jeremy\Anaconda\lib\site-packages\IPython\kernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
c:\Users\Jeremy\Anaconda\lib\site-packages\pandas\core\strings.py:207: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  " groups, use str.extract.", UserWarning)
c:\Users\Jeremy\Anaconda\lib\site-packages\IPython\kernel\__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on

In [38]:
# All reviews
subset[
    ['name', 'state', 'city', 'stars_y', 'attributes', 'categories_new', 'stars_x', 'text']
].rename(columns={
    'stars_y': 'business_stars', 'stars_x': 'review_stars', 'categories_new': 'categories'
}).to_csv('yelp_clean.csv', sep = '\t', index=False, quoting=csv.QUOTE_NONE)

In [37]:
# Charlotte
subset[
    ['name', 'state', 'city', 'stars_y', 'attributes', 'categories_new', 'stars_x', 'text']
].rename(columns={
    'stars_y': 'business_stars', 'stars_x': 'review_stars', 'categories_new': 'categories'
})[subset.city == 'Charlotte'].to_csv('yelp_charlotte_v2.csv', sep = '\t', index=False, quoting=csv.QUOTE_NONE)